### https://www.kaggle.com/CVxTz/audio-data-augmentation

In [72]:
import numpy as np
import random
import os
import itertools
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from multiprocessing import Process
%matplotlib inline

In [2]:
def plot_time_series(plot_data):
    fig = plt.figure(figsize=(7, 4))
    plt.title('Raw wave ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 1, len(plot_data)), plot_data)
    plt.show()

In [67]:
def read_wav(path):
    data, sr = librosa.core.load(path)
    # sr, data = wavfile.read(test_path)
    return data, sr


def save_wav(save_data, sr, path):
    wavfile.write(path, sr, save_data)

In [31]:
test_path = '/home/kolegor/test.wav'
data, sr = read_wav(test_path)
print len(data), sr

220500 22050


#### original

In [32]:
# plot_time_series(data)
ipd.Audio(data, rate=sr)

#### white noise

In [5]:
# Adding white noise 
wn = np.random.randn(len(data))
data_wn = data + 0.005*wn
# plot_time_series(data_wn)
ipd.Audio(data_wn, rate=sr)

#### shift

In [46]:
# Shifting the sound
# data_roll = np.roll(data, sr / 5)
data_roll = librosa.effects.pitch_shift(data, sr, -0.1)
# data_roll = librosa.effects.pitch_shift(data, sr, -0.3)
# plot_time_series(data_roll)
ipd.Audio(data_roll, rate=sr)

#### stretch

In [43]:
# stretching the sound
def stretch(file_data, rate=1):
    data = librosa.effects.time_stretch(file_data, rate)
    return data

In [44]:
data_stretch = stretch(data, 0.8)
print("This makes the sound deeper but we can still hear 'off' ")
# plot_time_series(data_stretch)
ipd.Audio(data_stretch, rate=sr)

This makes the sound deeper but we can still hear 'off' 


In [45]:
data_stretch = stretch(data, 1.2)
print("Higher frequencies  ")
# plot_time_series(data_stretch)
ipd.Audio(data_stretch, rate=sr)

Higher frequencies  


### Augment folder

In [63]:
def augment_white_noise(input_data, output_file, sr):
    wn = np.random.randn(len(input_data))
    data_wn = input_data + 0.005*wn
    save_wav(data_wn, sr, output_file)

def augment_stretch(input_data, output_file, sr, stretch_value):
    stretched = librosa.effects.time_stretch(input_data, stretch_value)
    save_wav(stretched, sr, output_file)

def augment_data_shift(input_data, output_file, sr, shift_value):
    shifted = librosa.effects.pitch_shift(input_data, sr, shift_value)
    save_wav(shifted, sr, output_file)

In [64]:
def get_folder_files(folder_path):
    return [u'{}/{}'.format(folder_path, filename) for filename in os.listdir(folder_path)]

In [85]:
def augment_files(wav_files_paths, uid=None):
    for i, file_path in enumerate(wav_files_paths):
        if i % 100 == 0:
            print ' -', uid, i, len(wav_files_paths)

        file_data, sr = read_wav(file_path)

        if i % 4 == 0:
            output_path = file_path + '_#AUG#WN.wav'
            augment_white_noise(file_data, output_path, sr)
        elif i % 4 == 1:
            output_path = file_path + '_#AUG#STRETCH_PLUS.wav'
            augment_stretch(file_data, output_path, sr, 0.8)
        elif i % 4 == 2:
            output_path = file_path + '_#AUG#STRETCH_MINUS.wav'
            augment_stretch(file_data, output_path, sr, 1.2)
        elif i % 4 == 3:
            output_path = file_path + '_#AUG#SHIFT.wav'
            augment_data_shift(file_data, output_path, sr, -0.1)


def augment_folder(folder_path, uid=None):
    all_files = get_folder_files(folder_path)
    augment_files(all_files, uid=uid)


def augment_folders(folders, uid=None):
    for folder in folders:
        print uid, folder
        augment_folder(folder, uid=uid)

In [86]:
all_folders = []
base_folder = '/home/kolegor/Study/Master/Work/data/clean'
for folder in os.listdir(base_folder):
    wavs_folder = '{}/{}/all/'.format(base_folder, folder)
    all_folders.append(wavs_folder)
print len(all_folders)

26


In [ ]:
processes = 4
procs = []

chunk_size = (len(all_folders) + processes - 1) / processes
for chunk in xrange(processes):
    chunk_folders = all_folders[chunk * chunk_size:min(len(all_folders), (chunk + 1) * chunk_size)]

    proc = Process(target=augment_folders, args=(chunk_folders, u'PROC # {}'.format(chunk)))
    procs.append(proc)
    proc.start()

# complete the processes
for proc in procs:
    proc.join()